In [ ]:
import random

class gen():
    # Constructor
    def __init__(self, inicio, fin, secuencia, organismo, funcion='desconocida') -> None:
        self.longitud = fin - inicio + 1 # Por slicing en python
        self.inicio = inicio
        self.fin = fin
        self.secuencia = secuencia.strip().upper() # todo en mayúsculas
        self.organismo = organismo
        self.funcion = funcion

    def codones(self, marco=1) -> list: # 1-3. 1 por default
        return [self.secuencia[i:i+3] for i in range(marco - 1, len(self.secuencia)-2, 3)] # -2 para que siempre regrese codones completos

    def codon_paro(self, codones_seq) -> bool:
        return any(codon in (codones_paro := ['UGA', 'UAG', 'UAA', 'TGA', 'TAG', 'TAA']) for codon in codones_seq) # si coinciden con alguno, sea DNA o RNA
    
    def contenido_gc(self, seq) -> float: # No usé el self.secuencia porque más adelante me conviene manejarlo de esta forma (en tRNA)
        return 100 * (seq.count('G') + seq.count('C')) / len(seq) # En porcentaje

    def longitud_seq(self, )
    


def hacer_seq() -> str:
    return ''.join(random.choice(['A', 'T', 'C', 'G']) for _ in range(34))

gen_prueba = gen(322, 456, (hacer_seq()), 'Humano')

codones_seq = gen_prueba.codones(2)
print(f'Secuencia del gen: {(seq := gen_prueba.secuencia)}\nCodones: {codones_seq}') # Segundo marco de lectura
print(f'Codón de paro: {gen_prueba.codon_paro(codones_seq)}\nContendo de GC: {gen_prueba.contenido_gc(seq)}%')

Secuencia del gen: AGAGGAGGCTATGATTTTTGACAGACACTTCATT
Codones: ['GAG', 'GAG', 'GCT', 'ATG', 'ATT', 'TTT', 'GAC', 'AGA', 'CAC', 'TTC', 'ATT']
Codón de paro: False
Contendo de GC: 38.23529411764706%


### tRNA

In [119]:
# Subclase ---> Ejercicio Herencia
class tRNA(gen):
    def __init__(self, anticodon, organismo='Desconocido') -> None:
        super().__init__(inicio=0, fin=len(anticodon) - 1, secuencia=anticodon, organismo=organismo)
        self.anticodon = anticodon
        self.aminoacido = None
        self.carga = False

    # Aquí está la función del ejercicio 4 de polimorfismo
    def len_seq(self) -> int:
        return len(self.secuencia)
    
    def generar_codon(self) -> str:
        bases_comp = {'A':'U','U':'A','C':'G','G':'C','T':'A'} # Tanto RNA como DNA, pero el resultado siempre en RNA
        return ''.join([bases_comp[base] for base in self.anticodon.strip().upper()])

    def identificar_aa(self, codon) -> str:
        codigo_genetico = {
            'F': ['UUU', 'UUC'], # Fenilalanina
            'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], # Leucina
            'I': ['AUU', 'AUC', 'AUA'], # Isoleucina
            'M': ['AUG'], # Metionina (Inicio)
            'V': ['GUU', 'GUC', 'GUA', 'GUG'], # Valina
            'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], # Serina
            'P': ['CCU', 'CCC', 'CCA', 'CCG'], # Prolina
            'T': ['ACU', 'ACC', 'ACA', 'ACG'], # Treonina
            'A': ['GCU', 'GCC', 'GCA', 'GCG'], # Alanina
            'Y': ['UAU', 'UAC'], # Tirosina
            'H': ['CAU', 'CAC'], # Histidina
            'Q': ['CAA', 'CAG'], # Glutamina
            'N': ['AAU', 'AAC'], # Asparagina
            'K': ['AAA', 'AAG'], # Lisina
            'D': ['GAU', 'GAC'], # Ácido aspártico
            'E': ['GAA', 'GAG'], # Ácido glutámico
            'C': ['UGU', 'UGC'], # Cisteína
            'W': ['UGG'], # Triptófano
            'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], # Arginina
            'G': ['GGU', 'GGC', 'GGA', 'GGG'], # Glicina
            '-STOP-': ['UAA', 'UAG', 'UGA'] # Codones de paro
        }
        for aa, codones in codigo_genetico.items():
            if codon in codones:
                self.carga = True
                return aa # Aminoácido asociado
        return None

    def descargar(self):
        self.carga = False   

In [120]:
trna = tRNA('UGC')
print(f'Anticodón ({trna.len_seq()} bases) del tRNA: {trna.anticodon}\nOrganismo: {trna.organismo}\nContenido de GC (si tuviera sentido): {trna.contenido_gc(codon := trna.generar_codon())}%\nAminoacido asociado: {trna.identificar_aa(codon)}\nCarga: {trna.carga}')

Anticodón (3 bases) del tRNA: UGC
Organismo: Desconocido
Contenido de GC (si tuviera sentido): 66.66666666666667%
Aminoacido asociado: T
Carga: True


#### Subclase Proteína

In [131]:
# Siento que es un poco inverosímil usar herencia en este contexto de tRNAs y proteínas, pero sería algo así:

class Proteina(tRNA):
    # Aprovechamos que hereda una función para separar secuencias en tripletes, y otra que los asocia con aminoácidos
    def __init__(self, secuencia, organismo='desconocido', funcion='desconocida') -> None: # Recordar que la secuenci debe ser de RNA
        gen.__init__(self, inicio=0, fin=len(secuencia)-1, secuencia=secuencia.replace('T','U'), organismo=organismo, funcion=funcion) # específicamente la clase gen
        self.seq_aa = None
        
    def traducir(self, marco=1) -> str:
        seq_aa = ''
        for codon in (codones := self.codones(marco)):
            seq_aa += self.identificar_aa(codon) # Añade cada Aminoácido en single letter code
        self.seq_aa = seq_aa # Lo guarda y lo regresa simplemente porque es más cómodo en el print
        return seq_aa # Quise usar walrus aquí y no me dejó :(

    # Ejercicio 4 ---> Polimorfismo
    def len_seq(self) -> int:
        return super().len_seq() , len(self.seq_aa.replace('-STOP-', '_')) # Para que no cuente los caracteres de más

    def funcion_estimada(self) -> str:
        funciones_por_longitud = { # todo esto es hipotético y obviamente no refleja un metodo de estimación de función real...
            (0, 50): 'péptido señal o regulador pequeño', # tuplas, que son iterables
            (51, 150): 'enzima pequeña o factor de unión a DNA/RNA',
            (151, 300): 'enzima típica, proteína estructural',
            (301, 500): 'proteína multimérica o complejo enzimático',
            (501, 1000): 'proteína grande, receptor o transportador',
            (1001, 5000): 'proteína gigante, complejo molecular, motor molecular'
        }

        for rango, funcion in funciones_por_longitud.items():
            if rango[0] <= (len_seq := len(self.seq_aa)) and rango[1] >= len_seq:
                return funcion
        return None # Si acabó y nunca estuvo en rango

In [132]:
beta_hemoglobina = Proteina('AUGGAUGAGAGAAUUAAGGAUGAAUGAUGAUGAUGAUAUAUGAGAUAUGAAAUUGAGAUUGAGAGGUAUUGAUAUUUAGGAGAGUUUUUUUUAGAGAGGAUAGUAGUUGAUGAUGAAUGAUGAUG', 'humano')

In [133]:
# Antes de usar los métodos
print(f'La beta-hemoglobina, del {beta_hemoglobina.organismo}, hasta ahora, tiene una función {beta_hemoglobina.funcion}')

La beta-hemoglobina, del humano, hasta ahora, tiene una función desconocida


In [135]:
seq_aa = beta_hemoglobina.traducir()
len_nt, len_aa = beta_hemoglobina.len_seq() # Aquí se ve la función, que ahora regresa ambos

In [136]:
print(f'Asimismo, la secuencia de aminoácidos de la proteína es:\n{seq_aa}\nCon base en su longitud de {len_nt} nucleótidos, y {len_aa} aminoácidos, su función putativa es ser {beta_hemoglobina.funcion_estimada()}')

Asimismo, la secuencia de aminoácidos de la proteína es:
MDERIKDE-STOP--STOP--STOP--STOP-YMRYEIEIERY-STOP-YLGEFFLERIVVDDE-STOP--STOP-
Con base en su longitud de 125 nucleótidos, y 41 aminoácidos, su función putativa es ser enzima pequeña o factor de unión a DNA/RNA


### ncRNA

In [99]:
import pandas as pd

In [117]:
class ncRNA(gen):
    def __init__(self, inicio, secuencia, organismo='Desconocido', tipo='Desconocido') -> None:
        super().__init__(inicio=inicio, fin=len(secuencia) - 1, secuencia=secuencia.replace('T','U'), organismo=organismo, funcion='ncRNA')
        self.tipo = tipo # miRNA, siRNA, lncRNA, snRNA, etc...
        self.len_seq = self.fin + 1
        self.contenido_gc = self.contenido_gc(self.secuencia)

    def puntuar(self, tipo, info, atributo, score) -> int:
        cantidad = self.len_seq if atributo == 'len' else self.contenido_gc
        return score if cantidad >= info[atributo][0] and cantidad <= info[atributo][1] else 0

    def predecir_tipo(self) -> str:

        heuristica = { # Pensadas con base en la longitud de la secuencia y su contenido de GC ---> No es biológicamente preciso...
            'miRNA': {'len': (18, 25), 'gc': (40, 60)},
            'siRNA': {'len': (20, 25), 'gc': (30, 55)},
            'piRNA': {'len': (24, 31), 'gc': (30, 60)},
            'snoRNA': {'len': (60, 200), 'gc': (45, 65)},
            'snRNA': {'len': (100, 300), 'gc': (45, 65)},
            'lncRNA': {'len': (200, 10**9), 'gc': (0, 100)},
            'rRNA': {'len': (1000, 10**9), 'gc': (40, 65)}
        }
        # Convertir a DataFrame para mejor manejo
        df_heuristica = pd.DataFrame.from_dict(heuristica, orient="index")

        # Sistema de puntuación ---> 2 puntos si coincide con el rango, 1 si coincide con el %GC
        for tipo, info in df_heuristica.iterrows():
            score = 0
            score += self.puntuar(tipo, info, 'len', 2)
            score += self.puntuar(tipo, info, 'gc', 1)
            df_heuristica.loc[tipo, 'score'] = score
        
        return df_heuristica.score.idxmax(), df_heuristica.score.max()

In [118]:
miRNA = ncRNA(220, 'GCUUAGGAUAAAUCUAGAGCCU')
tipo, score = miRNA.predecir_tipo()
print(f'El non-coding RNA de longitud {miRNA.len_seq} y %GC {miRNA.contenido_gc}, con un score de {score}, es putativamente un {tipo}')

El non-coding RNA de longitud 22 y %GC 40.90909090909091, con un score de 3.0, es putativamente un miRNA
